# Integer Constraints in Modeling: Building Realistic Discrete Models

This notebook demonstrates how to model **integer constraints** in optimization problems.

Understanding integer constraints is essential because:
- **Many business decisions require whole numbers** - people, machines, locations
- **Integer constraints fundamentally change how optimization works**
- **Binary variables (0/1) model yes/no decisions** - open/close, select/don't select
- **General integers model counts** - how many items, how many people
- **Integer models produce implementable recommendations**


## Key Concepts

**Integer Constraints** require variables to be whole numbers:
- Cannot have fractional values
- Examples: Number of people, machines, facilities
- Modeled with `cat='Integer'` in PuLP

**Binary Variables** are a special case of integers:
- Can only be 0 or 1
- Model yes/no decisions: open facility (1) or don't (0)
- Modeled with `cat='Binary'` in PuLP

**General Integer Variables** can be any whole number:
- 0, 1, 2, 3, ... up to some limit
- Model counts: how many machines, how many people

**Critical insight**: Integer constraints make problems harder to solve but produce realistic, implementable solutions.


## Scenario: Equipment Purchase Decision

You manage a manufacturing facility and need to decide:
1. **Which machines to purchase** (yes/no decision - binary)
2. **How many units of each machine** (count decision - integer)

**Available Machines**:
- Machine A: $50,000 each, produces 100 units/day
- Machine B: $80,000 each, produces 150 units/day
- Machine C: $120,000 each, produces 200 units/day

**Requirements**:
- Need at least 500 units/day production capacity
- Budget: $300,000
- Can purchase multiple units of the same machine

**The Challenge**: These are discrete decisions - you cannot buy 2.5 machines or "partially" purchase a machine.


## Step 1: Install Required Packages (Colab)


In [1]:
# Install pulp package (required for optimization)
# This is needed in Google Colab; can be skipped if already installed locally
%pip install pulp -q



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Set style for better-looking plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)


## Step 3: Define Problem Data


In [3]:
# Machine data
machines = ['Machine A', 'Machine B', 'Machine C']
costs = {'Machine A': 50000, 'Machine B': 80000, 'Machine C': 120000}
capacity = {'Machine A': 100, 'Machine B': 150, 'Machine C': 200}

# Requirements
min_production = 500  # Minimum units/day needed
budget = 300000  # Maximum budget

print("PROBLEM DATA:")
print("=" * 60)
for m in machines:
    print(f"{m}: Cost=${costs[m]:,}, Capacity={capacity[m]} units/day")
print(f"\nMinimum production needed: {min_production} units/day")
print(f"Budget limit: ${budget:,}")


PROBLEM DATA:
Machine A: Cost=$50,000, Capacity=100 units/day
Machine B: Cost=$80,000, Capacity=150 units/day
Machine C: Cost=$120,000, Capacity=200 units/day

Minimum production needed: 500 units/day
Budget limit: $300,000


## Step 4: Model with Integer Constraints

We'll use **integer variables** to model how many of each machine to purchase:


In [4]:
# Create optimization model
model = LpProblem("Equipment_Purchase", LpMinimize)

# Integer variables: how many of each machine to purchase
# These must be whole numbers (0, 1, 2, 3, ...)
purchase = {}
for m in machines:
    purchase[m] = LpVariable(f"purchase_{m.replace(' ', '_')}", 
                              lowBound=0, cat='Integer')

print("INTEGER VARIABLES CREATED:")
for m in machines:
    print(f"  {purchase[m].name}: Number of {m} to purchase (must be whole number)")

# Objective: Minimize total cost
model += lpSum([costs[m] * purchase[m] for m in machines]), "Total_Cost"

# Constraint: Must meet minimum production capacity
model += lpSum([capacity[m] * purchase[m] for m in machines]) >= min_production, "Min_Production"

# Constraint: Cannot exceed budget
model += lpSum([costs[m] * purchase[m] for m in machines]) <= budget, "Budget"

print("\nMODEL CREATED:")
print(f"  Objective: Minimize total cost")
print(f"  Constraint 1: Production >= {min_production} units/day")
print(f"  Constraint 2: Cost <= ${budget:,}")


INTEGER VARIABLES CREATED:
  purchase_Machine_A: Number of Machine A to purchase (must be whole number)
  purchase_Machine_B: Number of Machine B to purchase (must be whole number)
  purchase_Machine_C: Number of Machine C to purchase (must be whole number)

MODEL CREATED:
  Objective: Minimize total cost
  Constraint 1: Production >= 500 units/day
  Constraint 2: Cost <= $300,000


## Step 5: Solve the Integer Model


In [5]:
# Solve the model
model.solve()

# Get solution
solution = {}
for m in machines:
    solution[m] = int(value(purchase[m]))

total_cost = value(model.objective)
total_capacity = sum([capacity[m] * solution[m] for m in machines])

print("INTEGER MODEL SOLUTION:")
print("=" * 60)
for m in machines:
    if solution[m] > 0:
        print(f"  Purchase {solution[m]} units of {m}")
        print(f"    Cost: ${costs[m] * solution[m]:,}")
        print(f"    Capacity: {capacity[m] * solution[m]} units/day")

print(f"\nTotal Cost: ${total_cost:,.0f}")
print(f"Total Capacity: {total_capacity} units/day")
print(f"\n✓ All values are whole numbers - solution is implementable!")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/a7e2af2656914930919cd84fe29715e4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/a7e2af2656914930919cd84fe29715e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 23 RHS
At line 26 BOUNDS
At line 30 ENDATA
Problem MODEL has 2 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 250000 - 0.00 seconds
Cgl0004I processed model has 2 rows, 3 columns (3 integer (0 of which binary)) and 6 elements
Cutoff increment increased from 1e-05 to 10000
Cbc0012I Integer solution of 250000 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 secon

## Step 6: Binary Variables for Yes/No Decisions

Now let's model a different problem: **which facilities to open** (yes/no decision).

This requires **binary variables** (0 or 1):


In [6]:
# Facility location problem
facilities = ['Facility A', 'Facility B', 'Facility C', 'Facility D']
fixed_costs = {'Facility A': 100000, 'Facility B': 150000, 
               'Facility C': 120000, 'Facility D': 180000}
capacities = {'Facility A': 500, 'Facility B': 800, 
              'Facility C': 600, 'Facility D': 1000}

min_capacity_needed = 1000
budget_facilities = 400000

# Create model
model_binary = LpProblem("Facility_Location", LpMinimize)

# Binary variables: 1 = open facility, 0 = don't open
open_facility = {}
for f in facilities:
    open_facility[f] = LpVariable(f"open_{f.replace(' ', '_')}", 
                                  cat='Binary')

print("BINARY VARIABLES CREATED:")
for f in facilities:
    print(f"  {open_facility[f].name}: Open {f}? (1=Yes, 0=No)")

# Objective: Minimize total fixed cost
model_binary += lpSum([fixed_costs[f] * open_facility[f] for f in facilities]), "Total_Cost"

# Constraint: Must meet minimum capacity
model_binary += lpSum([capacities[f] * open_facility[f] for f in facilities]) >= min_capacity_needed, "Min_Capacity"

# Constraint: Budget limit
model_binary += lpSum([fixed_costs[f] * open_facility[f] for f in facilities]) <= budget_facilities, "Budget"

# Solve
model_binary.solve()

# Get solution
solution_binary = {}
for f in facilities:
    solution_binary[f] = int(value(open_facility[f]))

total_cost_binary = value(model_binary.objective)
total_capacity_binary = sum([capacities[f] * solution_binary[f] for f in facilities])

print("\nBINARY MODEL SOLUTION:")
print("=" * 60)
for f in facilities:
    status = "OPEN" if solution_binary[f] == 1 else "CLOSED"
    print(f"  {f}: {status} (value = {solution_binary[f]})")
    if solution_binary[f] == 1:
        print(f"    Cost: ${fixed_costs[f]:,}, Capacity: {capacities[f]}")

print(f"\nTotal Cost: ${total_cost_binary:,.0f}")
print(f"Total Capacity: {total_capacity_binary}")
print(f"\n✓ Binary variables produce clear yes/no decisions!")


BINARY VARIABLES CREATED:
  open_Facility_A: Open Facility A? (1=Yes, 0=No)
  open_Facility_B: Open Facility B? (1=Yes, 0=No)
  open_Facility_C: Open Facility C? (1=Yes, 0=No)
  open_Facility_D: Open Facility D? (1=Yes, 0=No)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/3c8a0641d07748aa9cf5e2d7a22176ed-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/3c8a0641d07748aa9cf5e2d7a22176ed-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 28 RHS
At line 31 BOUNDS
At line 36 ENDATA
Problem MODEL has 2 rows, 4 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 180000 - 0.00 seconds
Cgl0004I processed

## Summary: Integer Constraints in Modeling

**Integer Variables** (`cat='Integer'`):
- Can be any whole number: 0, 1, 2, 3, ...
- Model counts: how many machines, how many people
- Must be whole numbers, not fractional

**Binary Variables** (`cat='Binary'`):
- Can only be 0 or 1
- Model yes/no decisions: open/close, select/don't select
- Special case of integer variables

**When to Use Each**:
- Use **integer** for "how many" decisions
- Use **binary** for "which ones" or "yes/no" decisions

**Critical Takeaway**:
- Integer constraints make problems harder to solve computationally
- But they produce realistic, implementable solutions
- Always use integer constraints when decisions are discrete
- Don't round continuous solutions - model as integer from the start
